In [2]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import shutil, os, pathlib
import random

print(tf.__version__)

2024-07-30 14:03:33.542970: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 14:03:33.556029: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 14:03:33.634433: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 14:03:33.700245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 14:03:33.756251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

2.17.0


In [3]:
input_dir = "data/pets/images/"
target_dir = "data/pets/annotations/trimaps/"
input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)
target_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

In [4]:
img_size = (200, 200)
num_imgs = len(input_img_paths)
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_paths)


def path_to_input_image(path):
    return img_to_array(load_img(path, target_size=img_size))


def path_to_target(path):
    img = img_to_array(load_img(path, target_size=img_size, color_mode="grayscale"))
    img = img.astype("uint8") - 1
    return img


input_imgs = np.zeros((num_imgs,) + img_size + (3,), dtype="float32")
targets = np.zeros((num_imgs,) + img_size + (1,), dtype="uint8")
for i in range(num_imgs):
    input_imgs[i] = path_to_input_image(input_img_paths[i])
    targets[i] = path_to_target(target_paths[i])

print(input_imgs.shape)
print(targets.shape)

(7390, 200, 200, 3)
(7390, 200, 200, 1)


In [5]:
num_val_samples = 1000
train_input_imgs = input_imgs[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_input_imgs = input_imgs[-num_val_samples:]
val_targets = targets[-num_val_samples:]

In [6]:
class SegModel(tf.keras.Model):
    def __init__(self):
        super(SegModel, self).__init__()
        l1 = tf.keras.layers.Conv2D(64, 3, strides=2, activation="relu", padding="same")
        l2 = tf.keras.layers.Conv2D(64, 3, strides=1, activation="relu", padding="same")
        l3 = tf.keras.layers.Conv2D(
            128, 3, strides=2, activation="relu", padding="same"
        )
        l4 = tf.keras.layers.Conv2D(
            128, 3, strides=1, activation="relu", padding="same"
        )
        l5 = tf.keras.layers.Conv2D(
            256, 3, strides=2, activation="relu", padding="same"
        )
        l6 = tf.keras.layers.Conv2D(
            256, 3, strides=1, activation="relu", padding="same"
        )
        l7 = tf.keras.layers.Conv2DTranspose(
            256, 3, strides=1, activation="relu", padding="same"
        )
        l8 = tf.keras.layers.Conv2DTranspose(
            256, 3, strides=2, activation="relu", padding="same"
        )
        l9 = tf.keras.layers.Conv2DTranspose(
            128, 3, strides=1, activation="relu", padding="same"
        )
        l10 = tf.keras.layers.Conv2DTranspose(
            128, 3, strides=2, activation="relu", padding="same"
        )
        l11 = tf.keras.layers.Conv2DTranspose(
            64, 3, strides=1, activation="relu", padding="same"
        )
        l12 = tf.keras.layers.Conv2DTranspose(
            64, 3, strides=2, activation="relu", padding="same"
        )
        self.layers_list = [l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12]
        
    def call(self, inputs):
        z = inputs
        for l in self.layers_list:
            z = l(z)
        return z

model = SegModel()

: 

In [7]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("data/model_checkpoints/oxford_segmentation.keras", save_best_only=True)
]
history = model.fit(
    train_input_imgs,
    train_targets,
    epochs=50,
    callbacks=callbacks,
    batch_size=64,
    validation_data=(val_input_imgs, val_targets),
)